# Exercício de agentes inteligentes

Para este primeiro exercício, vamos criar um agente inteligente baseado no [aimacode](https://github.com/aimacode).

## Criando agentes

Inicialmente deveremos criar nossos agentes. Para isso, vamos criar uma classe genérica de uma coisa e um agente. 
A coisa, como o nome já diz, pode ser qualquer coisa.
O agente inicialmente esta vivo, não colidiu com nada, não segura nada, e tem sua performance zerada.
Um agente deve estar inserido em um ambiente para poder interagir, logo criaremos este também.

In [0]:
import collections

class Thing:
    def __repr__(self):
        return '<{}>'.format(getattr(self, '__name__', self.__class__.__name__))

    def is_alive(self):
        """Things that are 'alive' should return true."""
        return hasattr(self, 'alive') and self.alive

    def show_state(self):
        """Display the agent's internal state. Subclasses should override."""
        print("I don't know how to show_state.")

    def display(self, canvas, x, y, width, height):
        """Display an image of this Thing on the canvas."""
        # Do we need this?
        pass

class Agent(Thing):
    def __init__(self, program=None):
        self.alive = True
        self.bump = False
        self.holding = []
        self.performance = 0
        if program is None or not isinstance(program, collections.Callable):
            print("Can't find a valid program for {}, falling back to default.".format(
                self.__class__.__name__))

            def program(percept):
                return eval(input('Percept={}; action? '.format(percept)))

        self.program = program
        
class Environment:
    def __init__(self):
        self.things = []
        self.agents = []

    def thing_classes(self):
        return []  # List of classes that can go into environment

    def percept(self, agent):
        """Return the percept that the agent sees at this point. (Implement this.)"""
        raise NotImplementedError

    def execute_action(self, agent, action):
        """Change the world to reflect this action. (Implement this.)"""
        raise NotImplementedError

    def default_location(self, thing):
        """Default location to place a new thing with unspecified location."""
        return None

    def exogenous_change(self):
        """If there is spontaneous change in the world, override this."""
        pass

    def is_done(self):
        """By default, we're done when we can't find a live agent."""
        return not any(agent.is_alive() for agent in self.agents)

    def step(self):
        """Run the environment for one time step. If the
        actions and exogenous changes are independent, this method will
        do. If there are interactions between them, you'll need to
        override this method."""
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)
            self.exogenous_change()

    def run(self, steps=1000):
        """Run the Environment for given number of time steps."""
        for step in range(steps):
            if self.is_done():
                return
            self.step()

    def list_things_at(self, location, tclass=Thing):
        """Return all things exactly at a given location."""
        return [thing for thing in self.things
                if thing.location == location and isinstance(thing, tclass)]

    def some_things_at(self, location, tclass=Thing):
        """Return true if at least one of the things at location
        is an instance of class tclass (or a subclass)."""
        return self.list_things_at(location, tclass) != []

    def add_thing(self, thing, location=None):
        """Add a thing to the environment, setting its location. For
        convenience, if thing is an agent program we make a new agent
        for it. (Shouldn't need to override this.)"""
        if not isinstance(thing, Thing):
            thing = Agent(thing)
        if thing in self.things:
            print("Can't add the same thing twice")
        else:
            thing.location = location if location is not None else self.default_location(thing)
            self.things.append(thing)
            if isinstance(thing, Agent):
                thing.performance = 0
                self.agents.append(thing)

    def delete_thing(self, thing):
        """Remove a thing from the environment."""
        try:
            self.things.remove(thing)
        except ValueError as e:
            print(e)
            print("  in Environment delete_thing")
            print("  Thing to be removed: {} at {}".format(thing, thing.location))
            print("  from list: {}".format([(thing, thing.location) for thing in self.things]))
        if thing in self.agents:
            self.agents.remove(thing)

Agora criaremos o ambiente em que o nosso agente estará, que será um parque, o qual pode ter comida e água em qualquer parte dele. O parque será simplificado e terá apenas um caminho que o agente poderá seguir.

In [0]:
class Food(Thing):
    pass

class Water(Thing):
    pass

class Park(Environment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == "walk":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.walk()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog eat the first item
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        

    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

Agora vamos criar nosso primeiro agente. Ele será um cachorrinho que não enxerga muito bem, mas está com fome e com sede. 

In [0]:
class BlindDog(Agent):
    location = 1
    
    def walk(self):
        self.location += 1
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False


Agora implementaremos o programa que o agente inteligente utilizará. 
Ele irá controlar como o cachorrinho irá interagir sobre o ambiente, de acordo com a tabela abaixo:

| Percepção          | Ação    | 
|---------------------------|-------------|
| Sentir comida    |  Comer |
| Sentir água         | Beber   |
| Não sentir nada | Andar   |

In [0]:
def program(percepts):
    '''Returns an action based on the dog's percepts'''
    for p in percepts:
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'
    return 'walk'

Agora vamos rodar nossa simulação, criando o parque, com água e comida, além do nosso cachorrinho.

In [0]:
park = Park()
dog = BlindDog(program)
dogfood = Food()
water = Water()
park.add_thing(dog, 1)
park.add_thing(dogfood, 5)
park.add_thing(water, 7)

park.run(5)

BlindDog decided to walk at location: 1
BlindDog decided to walk at location: 2
BlindDog decided to walk at location: 3
BlindDog decided to walk at location: 4
BlindDog ate Food at location: 5


Veja que o cachorrinho andou 4 posições e na quinta parou e comeu.
Vamos continuar, para ver se ele encontra a água.

In [0]:
park.run(5)

BlindDog decided to walk at location: 5
BlindDog decided to walk at location: 6
BlindDog drank Water at location: 7


Ele encontrou! Vamos adicionar mais água no parque.

In [0]:
park.add_thing(water, 15)
park.run(10)

BlindDog decided to walk at location: 7
BlindDog decided to walk at location: 8
BlindDog decided to walk at location: 9
BlindDog decided to walk at location: 10
BlindDog decided to walk at location: 11
BlindDog decided to walk at location: 12
BlindDog decided to walk at location: 13
BlindDog decided to walk at location: 14
BlindDog drank Water at location: 15


Novamente ele conseguiu beber a água.

Agora é sua vez! Mude seu cachorrinho para que quando ele encontre um gato no parque, ele lata.

Para isso, você deve:

*   criar um gato (que pode ser uma coisa).
*   criar um nova classe de um cachorro (que pode extender da classe BlindDog) que late quando encontra um gato.
*   criar uma nova classe de um parque (que pode extender da classe Park) e modificar os métodos execute_action e is_done para executar as ações.


Além disso, rode a simulação novamente, adicionando um gato no parque, para testar se ele late quando encontra o gato.
Lembre-se que o cachorrinho deve continuar bebendo, comendo, além de latir ao encontrar um gato.

In [0]:
# Faça aqui seu agente e ambiente
class Cat(Thing):
    pass
 
class BlindBarkingDog(BlindDog):
  def bark(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Cat):
            return True
        return False
      
def program(percepts):
  '''Returns an action based on the dog's percepts'''
  for p in percepts:
      if isinstance(p, Food):
          return 'eat'
      elif isinstance(p, Water):
          return 'drink'
      elif isinstance(p, Cat):
          return 'bark'
  return 'walk'
      
class NewPark(Park):
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == "walk":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.walk()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog eat the first item
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "bark":
            items = self.list_things_at(agent.location, tclass=Cat)
            if len(items) != 0:
                if agent.bark(items[0]): #Have the dog drink the first item
                    print('{} barked to a {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.

                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) or isinstance(thing, Cat) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

Para testar se sua simulação está correta, utilize o código abaixo.

In [0]:
#%debug
park = NewPark()
dog = BlindBarkingDog(program)
dogfood = Food()
water = Water()
cat = Cat()
#%debug
park.add_thing(dog, 1)
park.add_thing(dogfood, 5)
park.add_thing(water, 7)
park.add_thing(cat, 12)

park.run(5)
park.run(5)
park.run(5)


BlindBarkingDog decided to walk at location: 1
BlindBarkingDog decided to walk at location: 2
BlindBarkingDog decided to walk at location: 3
BlindBarkingDog decided to walk at location: 4
BlindBarkingDog ate Food at location: 5
BlindBarkingDog decided to walk at location: 5
BlindBarkingDog decided to walk at location: 6
BlindBarkingDog drank Water at location: 7
BlindBarkingDog decided to walk at location: 7
BlindBarkingDog decided to walk at location: 8
BlindBarkingDog decided to walk at location: 9
BlindBarkingDog decided to walk at location: 10
BlindBarkingDog decided to walk at location: 11
BlindBarkingDog barked to a Cat at location: 12


Por último, indique quais os tipos do ambiente: 

Completamente ou parcialmente observável? Deterministico, estratégico ou estocástico? Episódico ou sequencial? Estático ou dinâmico? Discreto ou contínuo? Único agente ou multiagente?

*Informe aqui os tipos de ambiente*

 O ambiente é completamente observável, determinístico, sequencial, estático, discreto e contém um único agente.